### import libraries  

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random 
import pandas as pd 

### load the dataset  

In [3]:
chunk_iter = pd.read_csv('C:\\Users\\LENOVO\\Downloads\\aws_review_sofware_dataset\\aws_review_sofware_dataset.csv', chunksize=20000)   


In [5]:
df = next(chunk_iter) 

In [7]:
type(df) 

pandas.core.frame.DataFrame

### display the shape 


In [9]:
df.shape

(20000, 13)

### displaying top 5 rows 

In [11]:
df.head() 

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7.0,NaN
3,3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3.0,NaN
4,4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


### Display the column names of the DataFrame 

In [13]:
df.columns  

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [15]:
df['verified'].value_counts()/df.shape[0] 

verified
False    0.68415
True     0.31585
Name: count, dtype: float64

### Initializing empty lists for 'words' and 'sentences' columns  

In [17]:
df["words"]="default value"
df["sentences"]="default value"

for i in range(df.shape[0]):
    df.at[i,"words"]= []
    df.at[i,"sentences"] = [] 

### import nltk libraries  

In [19]:
import nltk
nltk.download('punkt_tab') 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Tokenize the review body into sentences 

In [21]:
from nltk.tokenize import sent_tokenize  

In [23]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1 

### Lemmatize each sentence 

In [25]:
#!pip install pywsd -q 
from pywsd.utils import lemmatize_sentence 

Warming up PyWSD (takes ~10 secs)... took 6.524877071380615 secs.


In [27]:
for k in range(df.shape[0]):
    df.at[k,"words"]=[]
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j])) 

In [29]:
df["words_sentences"] = "default" 

#### Combine all lemmatized words into a single string for each review 

In [31]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])  

 ### Feature Extraction -Applying CountVectorizer to the text data 

In [33]:
from sklearn.feature_extraction.text import  CountVectorizer 

In [36]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences) 

In [38]:
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out()) 

In [44]:
#df_y.sample() 

In [42]:
df_y=df["verified"] 

 ### Label Encoding 

In [46]:

from sklearn.preprocessing import LabelEncoder 

In [48]:

le=LabelEncoder() 

In [50]:
df_y_1=pd.DataFrame(df_y) 

In [52]:
df_y_enc=df_y_1.apply(le.fit_transform) 

In [54]:
df_y_enc.columns 

Index(['verified'], dtype='object')

In [56]:
df_y_enc.head(5) 

,verified
0,1
1,1
2,1
3,1
4,0


In [58]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y_enc, test_size=30, random_state=42) 

## Gradient Boosting Classifier ##  

In [60]:

from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=42)
GBC.fit(x_train, y_train)
y_pred=GBC.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = GBC.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_test,y_pred)*100)   

C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score : 77.1256885327992
accuracy_score :  83.33333333333334


## Navie Bayes Classifier ##    

In [66]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(x_train, y_train)
y_pred=GBC.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = MNB.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_test,y_pred)*100)  

C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score : 67.68152228342514
accuracy_score :  83.33333333333334


### Random Forest Classifier ### 

In [76]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=42)
RFC.fit(x_train, y_train)
y_pred=RFC.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = RFC.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score :", accuracy_score(y_pred,y_test)*100) 

C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Score : 99.93490235353029
accuracy_score : 80.0


## Logistic Regression with PCA ## 

In [84]:
#PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=1000)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test) 


In [86]:
from sklearn.linear_model import LogisticRegression 

In [88]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=21)
LR.fit(x_train, y_train)
y_pred=LR.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = LR.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_pred,y_test)*100) 


C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score : 79.17376064096145
accuracy_score :  70.0


In [90]:
#Logistic Regression with PCA
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=21)
LR.fit(x_train_pca, y_train)
y_pred=LR.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = LR.score(x_train_pca, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_pred,y_test)*100) 



C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score : 79.14872308462694
accuracy_score :  53.333333333333336


C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
